# Problem: Predicting Airplane Delays

The goals of this notebook are:
- Process and create a dataset from downloaded ZIP files
- Exploratory data analysis (EDA)
- Establish a baseline model and improve it

## Introduction to business scenario
You work for a travel booking website that is working to improve the customer experience for flights that were delayed. The company wants to create a feature to let customers know if the flight will be delayed due to weather when the customers are booking the flight to or from the busiest airports for domestic travel in the US. 

You are tasked with solving part of this problem by leveraging machine learning to identify whether the flight will be delayed due to weather. You have been given access to the a dataset of on-time performance of domestic flights operated by large air carriers. You can use this data to train a machine learning model to predict if the flight is going to be delayed for the busiest airports.

### Dataset
The provided dataset contains scheduled and actual departure and arrival times reported by certified US air carriers that account for at least 1 percent of domestic scheduled passenger revenues. The data was collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS). The dataset contains date, time, origin, destination, airline, distance, and delay status of flights for flights between 2014 and 2018.
The data are in 60 compressed files, where each file contains a CSV for the flight details in a month for the five years (from 2014 - 2018). The data can be downloaded from this [link](https://ucstaff-my.sharepoint.com/:f:/g/personal/ibrahim_radwan_canberra_edu_au/Er0nVreXmihEmtMz5qC5kVIB81-ugSusExPYdcyQTglfLg?e=bNO312). Please download the data files and place them on a relative path. Dataset(s) used in this assignment were compiled by the Office of Airline Information, Bureau of Transportation Statistics (BTS), Airline On-Time Performance Data, available with the following [link](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ). 

# Step 1: Prepare the environment 

Use one of the labs which we have practised on with the Amazon Sagemakers where you perform the following steps:
1. Start a lab.
2. Create a notebook instance and name it "oncloudproject".
3. Increase the used memory to 25 GB from the additional configurations.
4. Open Jupyter Lab and upload this notebook into it.
5. Upload the two combined CVS files (combined_csv_v1.csv and combined_csv_v2.csv), which you created in Part A of this project.

# Step 2: Build and evaluate simple models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use linear learner estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 

Note: You are required to perform the above steps on the two combined datasets separatey and to comments on the difference.

### 1. Load and Split Data
Load each dataset (`combined_csv_v1.csv` and the second dataset if you have it) and split it into training, validation, and testing sets.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("combined_csv_v1.csv")



/tmp/ipykernel_8205/2535875463.py:5: DtypeWarning: Columns (46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("combined_csv_v1.csv")


# Data Cleaning Script for NaN Values

This script identifies and removes rows with `NaN` values from a DataFrame, ensuring clean data for analysis.

## Steps:

1. **Check for NaN Values**  
   The script first checks for any `NaN` values in each column and prints the count of `NaN`s per column if they exist.

In [10]:
# Check for NaN values
nan_counts = df.isna().sum()
print("NaN values in each column:\n", nan_counts[nan_counts > 0])

# Remove rows with any NaN values
df.dropna(inplace=True)

# Verify that there are no NaN values left
print("\nNaN values after cleaning:\n", df.isna().sum().sum())  # Should output 0 if no NaNs are left

# Save the cleaned data back to the CSV
df.to_csv("combined_csv_v1_cleaned.csv", index=False)
print("Saved cleaned data to 'combined_csv_v1_cleaned.csv'")

NaN values in each column:
 target                  13946
DayOfWeek_2                 1
DayOfWeek_3                 1
DayOfWeek_4                 1
DayOfWeek_5                 1
DayOfWeek_6                 1
DayOfWeek_7                 1
Reporting_Airline_DL        1
Reporting_Airline_OO        1
Reporting_Airline_UA        1
Reporting_Airline_WN        1
Origin_CLT                  1
Origin_DEN                  1
Origin_DFW                  1
Origin_IAH                  1
Origin_LAX                  1
Origin_ORD                  1
Origin_PHX                  1
Origin_SFO                  1
Dest_CLT                    1
Dest_DEN                    1
Dest_DFW                    1
Dest_IAH                    1
Dest_LAX                    1
Dest_ORD                    1
Dest_PHX                    1
Dest_SFO                    1
DepHourofDay_1              1
DepHourofDay_2              1
DepHourofDay_4              1
DepHourofDay_5              1
DepHourofDay_6              1
DepHourofDay

# Data Preparation and Splitting Script

This script converts 'TRUE'/'FALSE' strings to Boolean values, splits the dataset into training, validation, and test sets, and saves each set to CSV files.


In [11]:
# Convert any 'TRUE'/'FALSE' strings to Boolean values if necessary
df.replace({'TRUE': True, 'FALSE': False}, inplace=True)

# Split features and target
X = df.drop(columns=['target'])  # Replace 'target' with the actual target column name if different
y = df['target']

# Split data into training (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split temp data into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Save these to CSV for uploading to S3
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
y_val.to_csv("y_val.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)


# S3 Bucket Creation Script

Creates an S3 bucket in the current AWS region, adjusting configuration for `us-east-1` and other regions.


In [12]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'flight-delay'  # Bucket name specified

# Get the current region
current_region = boto3.session.Session().region_name

# Create the bucket based on region
try:
    if current_region == 'us-east-1':
        # For us-east-1, we do not need to specify LocationConstraint
        response = s3.create_bucket(Bucket=bucket_name)
    else:
        # For other regions, include the LocationConstraint
        response = s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': current_region}
        )
    print(f"Bucket '{bucket_name}' created successfully in {current_region}!")
except Exception as e:
    print(f"Error creating bucket: {e}")


Bucket 'flight-delay' created successfully in us-east-1!


# S3 File Upload Script

Uploads training, validation, and test data files to the specified `flight-delay` S3 bucket under the `linear_learner` folder.


In [13]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'flight-delay'  # The existing bucket name

# Upload files to the flight-delay S3 bucket
for file in ["X_train.csv", "y_train.csv", "X_val.csv", "y_val.csv", "X_test.csv", "y_test.csv"]:
    s3.upload_file(file, bucket_name, f'linear_learner/{file}')
    print(f"Uploaded {file} to s3://{bucket_name}/linear_learner/{file}")


Uploaded X_train.csv to s3://flight-delay/linear_learner/X_train.csv
Uploaded y_train.csv to s3://flight-delay/linear_learner/y_train.csv
Uploaded X_val.csv to s3://flight-delay/linear_learner/X_val.csv
Uploaded y_val.csv to s3://flight-delay/linear_learner/y_val.csv
Uploaded X_test.csv to s3://flight-delay/linear_learner/X_test.csv
Uploaded y_test.csv to s3://flight-delay/linear_learner/y_test.csv


# S3 NaN Check Script

Downloads specified CSV files from the `flight-delay` S3 bucket, checks each file for `NaN` values, and prints results.


In [15]:
import pandas as pd
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'flight-delay'

# List of file names to check for NaN values
files = ["X_train.csv", "y_train.csv", "X_val.csv", "y_val.csv", "X_test.csv", "y_test.csv"]

# Function to check NaN values in each file
for file in files:
    # Download file from S3
    s3.download_file(bucket_name, f'linear_learner/{file}', file)
    
    # Load the file into a DataFrame
    df = pd.read_csv(file)
    
    # Check for NaN values
    nan_counts = df.isna().sum()
    nan_columns = nan_counts[nan_counts > 0]
    
    if not nan_columns.empty:
        print(f"\nNaN values found in {file}:")
        print(nan_columns)
    else:
        print(f"No NaN values in {file}")


No NaN values in X_train.csv
No NaN values in y_train.csv
No NaN values in X_val.csv
No NaN values in y_val.csv
No NaN values in X_test.csv
No NaN values in y_test.csv


# S3 Column Names Check Script

Downloads specified CSV files from the `flight-delay` S3 bucket, loads each file, and prints the column names for verification.


In [16]:
import pandas as pd
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'flight-delay'

# List of file names to check for column names
files = ["X_train.csv", "y_train.csv", "X_val.csv", "y_val.csv", "X_test.csv", "y_test.csv"]

# Function to check column names in each file
for file in files:
    # Download file from S3
    s3.download_file(bucket_name, f'linear_learner/{file}', file)
    
    # Load the file into a DataFrame
    df = pd.read_csv(file)
    
    # Print column names
    print(f"\nColumn names in {file}:")
    print(df.columns.tolist())



Column names in X_train.csv:
['Distance', 'Quarter_2', 'Quarter_3', 'Quarter_4', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5', 'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9', 'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13', 'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17', 'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21', 'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25', 'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29', 'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA', 'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW', 'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_

# Data Combination and Upload Script

Combines training and validation features with their respective targets, saves them as `train_combined.csv` and `val_combined.csv`, and uploads the files to the `flight-delay` S3 bucket.


In [17]:
import pandas as pd
import boto3

# Load training and validation features and targets, then combine them
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
train_combined = pd.concat([y_train, X_train], axis=1)  # Place target as the first column

X_val = pd.read_csv("X_val.csv")
y_val = pd.read_csv("y_val.csv")
val_combined = pd.concat([y_val, X_val], axis=1)

# Save combined files
train_combined.to_csv("train_combined.csv", index=False)
val_combined.to_csv("val_combined.csv", index=False)

# Upload to S3
s3 = boto3.client('s3')
bucket_name = 'flight-delay'
s3.upload_file("train_combined.csv", bucket_name, "linear_learner/train_combined.csv")
s3.upload_file("val_combined.csv", bucket_name, "linear_learner/val_combined.csv")
print("Combined files uploaded to S3.")


Combined files uploaded to S3.


# SageMaker Linear Learner Training Script

Sets up an AWS SageMaker session, prepares training and validation data from the `flight-delay` S3 bucket, and trains a binary classifier using the Linear Learner algorithm.



In [21]:
from sagemaker import LinearLearner
import boto3
import pandas as pd
import sagemaker

# Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = 'flight-delay'

# Download and load data
train_file = "train_combined.csv"
val_file = "val_combined.csv"
s3 = boto3.client('s3')
s3.download_file(bucket_name, f'linear_learner/{train_file}', train_file)
s3.download_file(bucket_name, f'linear_learner/{val_file}', val_file)

# Prepare training and validation data as numpy arrays
train_df = pd.read_csv(train_file)
val_df = pd.read_csv(val_file)
X_train_np = train_df.drop(columns=['target']).values.astype('float32')
y_train_np = train_df['target'].values.astype('float32')
X_val_np = val_df.drop(columns=['target']).values.astype('float32')
y_val_np = val_df['target'].values.astype('float32')

# Define Linear Learner estimator
linear = LinearLearner(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    predictor_type='binary_classifier',
    sagemaker_session=sagemaker_session,
)

# Convert to RecordSets
train_record_set = linear.record_set(X_train_np, labels=y_train_np)
val_record_set = linear.record_set(X_val_np, labels=y_val_np, channel="validation")

# Train the model by passing the RecordSets directly
linear.fit([train_record_set, val_record_set])


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2024-10-27-04-14-48-149


2024-10-27 04:14:49 Starting - Starting the training job...
2024-10-27 04:15:04 Starting - Preparing the instances for training...
2024-10-27 04:15:38 Downloading - Downloading input data...
2024-10-27 04:16:23 Downloading - Downloading the training image.........
2024-10-27 04:17:44 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/27/2024 04:17:49 INFO 139996902434624] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

# Model Deployment Script

Deploys the trained SageMaker Linear Learner model as an endpoint and retrieves the endpoint name for making predictions.



In [25]:
# Deploy the model to create an endpoint
linear_predictor = linear.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

# Get the endpoint name
model_name = linear_predictor.endpoint_name
print(f"Model deployed with endpoint name: {model_name}")


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: linear-learner-2024-10-27-04-30-59-933
INFO:sagemaker:Creating endpoint-config with name linear-learner-2024-10-27-04-30-59-933
INFO:sagemaker:Creating endpoint with name linear-learner-2024-10-27-04-30-59-933


--------!Model deployed with endpoint name: linear-learner-2024-10-27-04-30-59-933


# Test Data Cleaning Script

Cleans the `X_test_clean.csv` dataset by converting all values to numeric, removing rows with `NaN` values, and ensuring the correct number of features (93). The cleaned data is saved as `X_test_final.csv` without headers or index.


In [35]:
import pandas as pd

# Load the test CSV without headers to avoid accidental misinterpretation
X_test = pd.read_csv("X_test_clean.csv", header=None)

# Check for non-numeric data and ensure all values are numeric
X_test = X_test.apply(pd.to_numeric, errors='coerce')  # Convert to NaN if non-numeric

# Remove rows with NaN values
X_test.dropna(inplace=True)

# Ensure consistent row length matches expected number of features
expected_columns = 93  # Adjust this number based on your model's input features
if X_test.shape[1] != expected_columns:
    print(f"Error: Expected {expected_columns} columns, but found {X_test.shape[1]}.")
else:
    # Save the cleaned data without headers and index
    X_test.to_csv("X_test_final.csv", index=False, header=False)
    print("Cleaned data saved as X_test_final.csv.")


/tmp/ipykernel_8205/242272886.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("X_test_clean.csv", header=None)


Cleaned data saved as X_test_final.csv.


# S3 Upload Script for Cleaned Test Data

Uploads the cleaned test file `X_test_final.csv` to the `flight-delay` S3 bucket under the `linear_learner` folder.


In [36]:
import boto3

# Upload the cleaned test file to S3
s3 = boto3.client('s3')
s3.upload_file("X_test_final.csv", "flight-delay", "linear_learner/X_test_final.csv")
print("Uploaded cleaned test file to S3.")


Uploaded cleaned test file to S3.


# Batch Transformation Script

Performs a batch transformation on `X_test_final.csv` using the deployed model `linear-learner-2024-10-27-04-30-59-933`, with output saved to the specified S3 path.


In [37]:
from sagemaker.transformer import Transformer

# Initialize the transformer with the correct model endpoint
transformer = Transformer(
    model_name="linear-learner-2024-10-27-04-30-59-933",
    instance_count=1,
    instance_type="ml.m5.large",
    output_path="s3://flight-delay/linear_learner/output"
)

# Run the batch transform job with the final cleaned test data
transformer.transform("s3://flight-delay/linear_learner/X_test_final.csv", content_type="text/csv", split_type="Line")
transformer.wait()

print("Batch transformation completed. Results saved to S3.")


INFO:sagemaker:Creating transform job with name: linear-learner-2024-10-27-05-02-23-378


...................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
[10/27/2024 05:08:13 INFO 140357897824064] Memory profiler is not enabled by the environment variable ENABLE_PROFILER.
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
[10/27/2024 05:08:18 WARNING 140357897824064] Loggers have already been setup.
[10/27/2024 05:08:18 INFO 140357897824064] loaded entry point class algorithm.serve.server_config:config_api
[10/27/2024 05:08:18 INFO 140357897824064] loading entry points
[10/27/2024 05:08:18 I

# List S3 Output Folder Contents

Lists all objects in the `linear_learner/output/` folder of the `flight-delay` S3 bucket, displaying each file's key.


In [41]:
# List objects in the output folder
s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket='flight-delay', Prefix='linear_learner/output/')
for obj in response.get('Contents', []):
    print(obj['Key'])


linear_learner/output/X_test_final.csv.out


# Download Predictions File

Downloads the predictions file `X_test_final.csv.out` from the `flight-delay` S3 bucket and saves it locally as `predictions.csv`.


In [44]:
# Replace 'X_test.csv.out' with the correct file name if necessary
s3.download_file('flight-delay', 'linear_learner/output/X_test_final.csv.out', 'predictions.csv')


# Evaluation Script for Model Predictions

This script downloads the prediction results from S3, aligns them with the actual test labels, and calculates key performance metrics such as accuracy and a classification report.



In [49]:
import pandas as pd
import json
from sklearn.metrics import accuracy_score, classification_report
import boto3

# Define the S3 path for predictions and download them
output_key = 'linear_learner/output/X_test_final.csv.out'
s3 = boto3.client('s3')
s3.download_file('flight-delay', output_key, 'predictions.csv')

# Load predictions, parsing JSON if needed
with open('predictions.csv', 'r') as f:
    predictions = pd.Series([json.loads(line)['predicted_label'] for line in f])

# Load the actual labels
y_test = pd.read_csv('y_test.csv', header=None).squeeze()

# Ensure both y_test and predictions are of numeric type
y_test = pd.to_numeric(y_test, errors='coerce')
predictions = pd.to_numeric(predictions, errors='coerce')

# Drop any rows where conversion failed (if any NaNs were introduced by conversion)
y_test = y_test.dropna().reset_index(drop=True)
predictions = predictions.dropna().reset_index(drop=True)

# Align lengths
min_length = min(len(y_test), len(predictions))
y_test_aligned = y_test.iloc[:min_length]
predictions_aligned = predictions.iloc[:min_length]

# Calculate and display performance metrics
print("Test Accuracy:", accuracy_score(y_test_aligned, predictions_aligned))
print(classification_report(y_test_aligned, predictions_aligned))


Test Accuracy: 0.7790692496103317
              precision    recall  f1-score   support

         0.0       0.78      0.99      0.88    105531
         1.0       0.22      0.01      0.01     29199

    accuracy                           0.78    134730
   macro avg       0.50      0.50      0.44    134730
weighted avg       0.66      0.78      0.69    134730



# Final Comments

The model achieved an overall test accuracy of **77.9%**. The performance metrics for each class are as follows:

- **Class 0 (Non-target)**: High precision (0.78) and recall (0.99), indicating that the model is highly effective at identifying this majority class.
- **Class 1 (Target)**: Lower precision (0.22) and very low recall (0.01), suggesting the model struggles to identify this minority class accurately.

### Observations:

- The **imbalance between classes** may be affecting the model's ability to generalize well for Class 1. This is reflected in the lower f1-score and recall for this class.
- The **macro average** f1-score (0.44) highlights the uneven performance across classes, while the **weighted average** (0.69) reflects the model's bias towards the majority class.

### Recommendations:

To improve Class 1 predictions, consider:
- **Balancing the dataset** through techniques like oversampling or synthetic data generation for the minority class.
- **Adjusting the model** to account for class imbalance, such as using class weights or exploring ensemble methods that could enhance minority class prediction.
